In [16]:
#Import Packages
import pandas as pd
import datetime
from ErcotApi import ercot_connection
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

DA and RT Prices

In [15]:
#ERCOT Prices Calls
dtfrom = datetime.date.today() + datetime.timedelta(days=-6) 
dtto = datetime.date.today() + datetime.timedelta(days=-6) 

#hubs_lz = ["HB_NORTH", "HB_HOUSTON", "HB_SOUTH", "HB_WEST", "LZ_NORTH", "LZ_HOUSTON", "LZ_SOUTH", "LZ_WEST"]
hubs_lz = ["HB_NORTH"]

rtm_url = ["np6-905-cd/spp_node_zone_hub?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPointType=HU&size=1000",
           "np6-905-cd/spp_node_zone_hub?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPointType=LZ&size=1000"]

da_url = ["np4-190-cd/dam_stlmnt_pnt_prices?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPoint=HB_NORTH&size=1000"]


rtm_hublz = ercot_connection(rtm_url)
da_hublz = ercot_connection(da_url)

#ERC Prices Data Clean
def ercot_prices(price_data, market):
    prices = pd.DataFrame()
    for df in price_data:     
        
        if  market.upper() == "RT":
            spp =df[df["settlementPoint"]!= "HB_PAN"]
            #spp = df
            #spp  = df.groupby(["deliveryDate", "deliveryHour", "settlementPoint"])["settlementPointPrice"].mean().reset_index()
            spp["market"] = "RT"
        else:
            df["market"] = "DA"
            spp = df.rename({"hourEnding":"deliveryHour"}, axis=1)
            spp["deliveryHour"] = pd.to_numeric(spp["deliveryHour"].str.split(":").str[0])
            spp = spp.drop('DSTFlag', axis=1)
            
        deliveryHour = [datetime.time(num-1).strftime("%H:00:00") for num in spp["deliveryHour"]]
        spp["DateTime"] = pd.to_datetime(spp["deliveryDate"] + " "+ deliveryHour, format = "%Y-%m-%d %H:%M:%S")
        spp["type"] = spp["settlementPoint"].str.split("_").str[0]

        prices = pd.concat([prices, spp], ignore_index=True)
    return(prices)
rt_hub_lz = ercot_prices(rtm_hublz, "RT")
